In [2]:
import cv2

In [3]:
import numpy as np
from numba import cuda
import time
import math

In [4]:
#GPU function
@cuda.jit
def process_gpu(img,channels):
    tx = cuda.blockIdx.x*cuda.blockDim.x+cuda.threadIdx.x
    ty = cuda.blockIdx.y*cuda.blockDim.y+cuda.threadIdx.y
    for c in range(channels):
        color = img[tx,ty][c]*2.0+30
        if color > 255:
            img[tx,ty][c]=255
        elif color < 0:
            img[tx,ty][c] = 0
        else :
            img[tx,ty][c]= color
        

In [5]:
#CPU function
def process_cpu(img,dst):
    rows,cols,channels=img.shape
    for i in range(rows):
        for j in range(cols):
            for c in range(3):
                color=img[i,j][c]*2.0+20
                if color>255:
                    dst[i,j][c]=255
                elif color<0:
                    dst[i,j][c]=0
                else:
                    dst[i,j][c]=color

In [7]:
if __name__ == "__main__":
    #Create an image
    img = cv2.imread('/home/richard/图片/Figure_1.png')
    rows,cols,channels=img.shape
    dst_cpu = img.copy()
    dst_gpu = img.copy()
    
    
    start_cpu = time.time()
    process_cpu(img,dst_cpu)
    end_cpu = time.time()
    cpu_time = end_cpu-start_cpu
    print("CPU process time: "+str(cpu_time))
    
    
    ##GPU function
    dImg = cuda.to_device(img)
    threadsperblock = (16,16)
    blockspergrid_x = int(math.ceil(rows/threadsperblock[0]))
    blockspergrid_y = int(math.ceil(cols/threadsperblock[1]))
    blockspergrid = (blockspergrid_x,blockspergrid_y)
    cuda.synchronize()
    start_gpu = time.time()
    process_gpu[blockspergrid,threadsperblock](dImg,channels)
    cuda.synchronize()
    end_gpu = time.time()
    dst_gpu = dImg.copy_to_host()
    time_gpu = end_gpu-start_gpu
    
    cv2.imwrite("result_cpu.jpg",dst_cpu)
    cv2.imwrite("result_gpu.hpg",dst_gpu)
    print("Done.")
    
    

CPU process time: 36.58888077735901


CudaSupportError: Error at driver init: 
[100] Call to cuInit results in CUDA_ERROR_NO_DEVICE: